In [1]:
import os

ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
USERS_INFO_CSV = os.path.join(ROOT_DIR, 'input', 'users-info.csv')


NameError: name '__file__' is not defined

In [ ]:
USERS_INFO_CSV